In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import re

In [133]:
zones = ["", "China","emerg","Europe","Global","India","Japan","Rest"]

def process_df(x):
    columns = x.iloc[0,:]
    columns.name = ""
    x.columns = columns
    x = x.iloc[1:,:]
    x.reset_index(drop=True, names=columns, inplace=True)
    x.rename(index={"6":""}).head(7)
    return x

def get_betas(zone=""):
    path = "data_damodaran/discount_rate_estimation/beta_damodaran/totalbeta" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_unleveraged_betas(zone=""):
    path = "data_damodaran/discount_rate_estimation/beta_leverage/beta" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_taxrates(zone=""):
    path = "data_damodaran/discount_rate_estimation/tax/tax_rate_country/taxrate" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry name")
    return x

def get_wacc(zone=""):
    path = "data_damodaran/discount_rate_estimation/wacc_damodaran/wacc" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_spread_classifier(size=5e9, financial=False):
    path = "data_damodaran/capital_structure/spread_classifiers/ratings.xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Start here Ratings sheet"]
    moody = process_df(x.iloc[16:32,:4])
    moody_financefirms = process_df(x.iloc[16:,5:9])
    moody_smallfirms = process_df(x.iloc[35:,:4])
    
    if financial == True:
        return moody_financefirms
    else:
        if size >= 5e9:
            return moody
        else: 
            return moody_smallfirms
        
        
def get_10y_tbills():
    path = "data_damodaran/discount_rate_estimation/treasury_bills/histretSP.xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["T. Bond yield & return"]
    x = x.iloc[5:,:]
    x = process_df(x)
    return x

def get_moody_spread(x, size=5e9, financial=False):
    for i in range(len(get_spread_classifier(size,financial=financial))):
        if x <= get_spread_classifier(size,financial=financial).iloc[i,1]:
            return get_spread_classifier(size,financial=financial).iloc[i,-1]
        else:
            continue
    return get_spread_classifier(size,financial=financial).iloc[-1,-1]

In [134]:
x = yf.Ticker("NVDA")
x.info.get("industry")

sector = "Semiconductor"

### Capital Structure

In [149]:
E = x.balancesheet.loc["Stockholders Equity",:].iloc[0]
D = x.balancesheet.loc["Total Liabilities Net Minority Interest",:].iloc[0]
print("Debt:",D,"Equity:",E)

Debt: 19081000000.0 Equity: 22101000000.0


### free-risk rate

In [140]:
get_10y_tbills()
rf = 0.02

### Rd Damodaran Estimation

In [141]:
rd = get_wacc().loc[sector,"Cost of Debt"]

In [142]:
interest = x.incomestmt.loc["Interest Expense",:]
ebit = x.incomestmt.loc["EBIT",:]
coverage_ratio = ebit / interest

coverage_ratio.apply(get_moody_spread)

spread = get_moody_spread(coverage_ratio.iloc[-1])
spread

rd = rf + spread
rd 

0.0274

### Re Damodaran Estimation

In [143]:
re = get_wacc().loc[sector,"Cost of Equity"]

In [153]:
### 
tax_rate = get_taxrates("").loc[sector, "Average across only money-making companies"]
beta_u = get_unleveraged_betas().loc[sector,"Unlevered beta"]
beta = beta_u * (1 + (1-tax_rate)*D/E)

In [155]:
beta_u * (1 + (1-tax_rate)*D/E)


2.631685500109736

In [156]:
get_unleveraged_betas().loc[sector:,"Unlevered beta"]

Industry Name
Semiconductor                        1.482488
Semiconductor Equip                  1.614426
Shipbuilding & Marine                0.730087
Shoe                                 1.243617
Software (Entertainment)             1.316169
Software (Internet)                  1.369908
Software (System & Application)      1.373314
Steel                                1.105414
Telecom (Wireless)                   0.692552
Telecom. Equipment                   1.133812
Telecom. Services                    0.468573
Tobacco                              1.694705
Transportation                       0.866489
Transportation (Railroads)           0.918688
Trucking                             1.162314
Utility (General)                    0.408105
Utility (Water)                      0.869437
Total Market                         0.825787
Total Market (without financials)    1.079154
Name: Unlevered beta, dtype: object